In [20]:
import pandas as pd
import configs
import json
import utilities as utils
from PIL import Image
import importlib 
importlib.reload(utils)
importlib.reload(configs)
from cap_from_youtube import cap_from_youtube


In [3]:
df = pd.read_csv(f'{configs.DATASET_DIR}/validate.csv')

In [4]:
with open(f"{configs.DATASET_DIR}/validate.json") as f:
    val_dict = json.load(f)

In [14]:
url = val_dict["--07WQ2iBlw"]["url"]

In [21]:
cap = cap_from_youtube(url)

[youtube] Extracting URL: https://www.youtube.com/watch?v=--07WQ2iBlw
[youtube] --07WQ2iBlw: Downloading webpage
[youtube] --07WQ2iBlw: Downloading android player API JSON
[youtube] --07WQ2iBlw: Downloading player c353919c


In [22]:
cap

< cv2.VideoCapture 0x10d7e1650>

TORCHVISION ATTEMPT

In [30]:
import torchvision

In [31]:
kinetics_dataset = torchvision.datasets.Kinetics(root = "", frames_per_clip = 1, step_between_clips =1, )

TypeError: Kinetics.__init__() missing 1 required positional argument: 'root'

VIDEOMAE

In [25]:
import av
import torch
import numpy as np

from transformers import AutoImageProcessor, VideoMAEForVideoClassification
from huggingface_hub import hf_hub_download

np.random.seed(0)

/Users/ameyavaidya/opt/anaconda3/envs/cos429-final/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

In [27]:
def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

In [28]:
# video clip consists of 300 frames (10 seconds at 30 FPS)
file_path = hf_hub_download(
    repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
)
container = av.open(file_path)

In [ ]:
# sample 16 frames
indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
model = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")

In [8]:
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# model predicts one of the 400 Kinetics-400 classes
predicted_label = logits.argmax(-1).item()


NameError: name 'model' is not defined